In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/home/luban/anaconda3/envs/python36/bin/python'

import findspark
findspark.init()

from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import col,udf,date_sub,explode,split,date_add
from pyspark.sql.types import StringType,DoubleType,IntegerType
from pyspark.sql import functions as F
app_name = 'liuyuhang'
conf = (SparkConf().setAppName(app_name)
        .set('spark.master', 'yarn')
        .set('spark.submit.deployMode', 'client')
        .set('spark.driver.memory',                  '20g')
        .set('spark.executor.memory',                '8g')
        .set('spark.dynamicAllocation.minExecutors', '50')
        .set('spark.dynamicAllocation.maxExecutors', '200')
        .set('spark.yarn.queue',                     'root.pricing_dape_core_service_prod')
        .set('spark.ui.port','8060')
        .set('spark.executor.memoryOverhead','4096')
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
        .set('spark.driver.allowMultipleContexts', 'True')
        .set('spark.yarn.dist.archives','hdfs://DClusterNmg4/user/bigdata_driver_ecosys_test/ly/python_env/python3.6.2.tgz#python3.6.2')
        .set('spark.pyspark.driver.python',          '/home/luban/anaconda3/envs/python36/bin/python')
        .set('spark.pyspark.python', './python3.6.2/bin/python')
        .set("spark.yarn.appMasterEnv.PYSPARK_PYTHON", './python3.6.2/bin/python')
        )
spark = SparkSession.builder.config(conf = conf).enableHiveSupport().getOrCreate()
sc = spark.sparkContext
hc = HiveContext(sc)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('expand_frame_repr',False)

In [32]:
# 乘客行为表
df = spark.sql('''
select
    city_id,
    sum(bubble_cnt) as bubble_cnt,
    --冒泡数
    sum(if(pas_active_tag = 2, bubble_cnt, 0)) as high_freq_bubble_cnt,
    --中高活跃用户冒泡数
    sum(if(pas_active_tag = 1, bubble_cnt, 0)) as low_freq_bubble_cnt,
    --低活跃用户冒泡数
    sum(if(pas_active_tag = 0, bubble_cnt, 0)) as xcl_freq_bubble_cnt,
    --新沉流用户冒泡数
    sum(call_cnt) as call_cnt,
    --发单数
    sum(if(pas_active_tag = 2, call_cnt, 0)) as high_freq_call_cnt,
    --中高活跃用户发单数
    sum(if(pas_active_tag = 1, call_cnt, 0)) as low_freq_call_cnt,
    --低活跃用户发单数
    sum(if(pas_active_tag = 0, call_cnt, 0)) as xcl_freq_call_cnt,
    --新沉流用户发单数
    sum(success_cnt) as success_cnt,
    --完单数
    sum(if(pas_active_tag = 2, success_cnt, 0)) as high_freq_success_cnt,
    --中高活跃用户完单数
    sum(if(pas_active_tag = 1, success_cnt, 0)) as low_freq_success_cnt,
    --低活跃用户完单数
    sum(if(pas_active_tag = 0, success_cnt, 0)) as xcl_freq_success_cnt,
    --新沉流用户完单数
    sum(gmv) as gmv,
    --完单GMV
    sum(if(pas_active_tag = 2, gmv, 0)) as high_freq_gmv,
    --中高活跃用户完单GMV
    sum(if(pas_active_tag = 1, gmv, 0)) as low_freq_gmv,
    --低活跃用户完单GMV
    sum(if(pas_active_tag = 0, gmv, 0)) as xcl_freq_gmv,
    --新沉流用户完单GMV
    sum(driver_divide) as driver_divide,
    --司机抽成
    sum(business_profit) as business_profit,
    --毛利额
    sum(subsidy_hufan) as subsidy_hufan,
    --完单呼返
    sum(subsidy_c) as subsidy_c,
    --完单C补
    sum(subsidy_b) as subsidy_b, --完单B补
    dt,
    product_name
from
    (
        select
            passenger_id,
            dt,
            city_id,
            case
                when product_line = '特惠自营' then '特惠快车'
                when product_line = '普通快车非拼' then '普通快车'
                when product_line = '市内拼车' then '市内拼车'
                when product_line = '专车' then '专车'
                when product_line = '出租车' then '出租车'
                when product_line = '网约车' then '网约车'
                when product_line = '网+出' then '网+出'
                when product_line = '新泛快' then '新泛快'
                when product_line = '泛快大盘' then '泛快大盘'
            end as product_name,
            case
                when product_line = '网+出' then '网+出'
                when product_line = '出租车' then '网+出'
                else '网约车'
            end as pas_product_name,
            sum(bubble_cnt) as bubble_cnt,
            --冒泡数
            sum(objective_call_cnt) as call_cnt,
            --发单数
            sum(suc_cnt) as success_cnt,
            --完单数
            sum(gmv) as gmv,
            --完单GMV
            sum(online_driver_divide) as driver_divide,
            --司机抽成
            sum(business_profit) as business_profit,
            --毛利额
            sum(subsidy_hufan) as subsidy_hufan,
            --完单呼返
            sum(subsidy_c) as subsidy_c,
            --完单C补
            sum(subsidy_b) as subsidy_b --完单B补
        from
            mp_data.dwm_trip_trd_psg_core_di
        where
            dt = '2023-04-01'
            and product_line in (
                '特惠自营',
                --特惠快车
                '普通快车非拼',
                --普通快车
                '市内拼车',
                --市内拼车
                '专车',
                --专车
                '出租车',
                --出租车
                '网约车',
                --网约车
                '网+出',
                --网+出
                '新泛快',
                --新泛快
                '泛快大盘' --泛快大盘
            )
            and city_id in (1, 42, 366)
        group by
            passenger_id,
            dt,
            city_id,
            product_line
    ) a
    left join (
        select
            passenger_id,
            product_line as pas_product_name,
            case
                when pas_call_active_tag in (3, 4) then 2 --中高活跃用户
                when pas_call_active_tag = 2 then 1 --低活跃用户
                else 0 --新沉流用户
            end as pas_active_tag
        from
            dd_cdm.dm_trip_pas_label_active_df
        where
            dt = '2023-04-01'
            and product_line in ('网约车', '网+出')
            and city_id in (1, 42, 366)
    ) b on a.passenger_id = b.passenger_id
    and a.pas_product_name = b.pas_product_name
group by
    dt,
    city_id,
    product_name
order by
    city_id, product_name
limit
    100
''').toPandas()
df

,city_id,bubble_cnt,high_freq_bubble_cnt,low_freq_bubble_cnt,xcl_freq_bubble_cnt,call_cnt,high_freq_call_cnt,low_freq_call_cnt,xcl_freq_call_cnt,success_cnt,high_freq_success_cnt,low_freq_success_cnt,xcl_freq_success_cnt,gmv,high_freq_gmv,low_freq_gmv,xcl_freq_gmv,driver_divide,business_profit,subsidy_hufan,subsidy_c,subsidy_b,dt,product_name
0,1,1535282.000000,1009596.000000,292586.000000,233099.000000,115311.000000,97794.000000,12211.000000,5306.000000,94085.000000,80363.000000,9873.000000,3849.000000,6455303.740000,5465229.700000,700575.470000,289498.570000,4980189.880000,1045939.840298,51944.410000,142001.945500,8549.360000,2023-03-05,专车
1,1,1790653.000000,1189207.000000,339540.000000,261903.000000,218703.000000,159833.000000,37814.000000,21056.000000,178820.000000,133214.000000,30401.000000,15205.000000,9663313.700926,6811069.645644,1715390.716371,1136853.338911,7166291.999503,-5000.937678,0.000000,6953.770000,1154.349035,2023-03-05,出租车
2,1,1661306.000000,1209728.000000,273218.000000,178357.000000,153440.000000,121948.000000,20143.000000,11349.000000,109692.000000,87531.000000,14496.000000,7665.000000,3786047.370000,3036605.270000,485840.740000,263601.360000,2161771.629191,168569.826591,0.000000,1069006.539100,258371.765484,2023-03-05,市内拼车
3,1,4475117.000000,3047067.000000,797065.000000,630979.000000,1060287.000000,792374.000000,170347.000000,97566.000000,898494.000000,680967.000000,142817.000000,74710.000000,29587468.510000,22323667.190000,4690699.440000,2573101.880000,22375366.710000,2535107.794735,248202.380000,1820191.448400,1633009.141907,2023-03-05,新泛快
4,1,2349178.000000,1588232.000000,426923.000000,334019.000000,772007.000000,572656.000000,124980.000000,74371.000000,652617.000000,491675.000000,104479.000000,56463.000000,20911421.070000,15679388.530000,3353097.110000,1878935.430000,16177237.440000,1717102.116655,20998.090000,918411.127700,1220582.673794,2023-03-05,普通快车
5,1,2544944.000000,1730388.000000,456997.000000,357555.000000,1306850.000000,994147.000000,199735.000000,112968.000000,1086419.000000,835612.000000,165131.000000,85676.000000,36575521.800000,28078492.680000,5517212.910000,2979816.210000,27028056.069191,3027918.290501,248202.380000,2973823.840300,2057084.261631,2023-03-05,泛快大盘
6,1,1847975.000000,1242731.000000,329636.000000,275606.000000,278459.000000,207449.000000,45529.000000,25481.000000,222504.000000,169517.000000,35669.000000,17318.000000,7887774.140000,5986370.290000,1242377.120000,659026.730000,5575361.450000,778502.800773,227204.290000,855729.774900,365385.037297,2023-03-05,特惠快车
7,1,2787270.000000,1956948.000000,488015.000000,342303.000000,1647582.000000,1269996.000000,243390.000000,134196.000000,1370704.000000,1067535.000000,201387.000000,101782.000000,53883833.370926,41660319.645644,7871189.836371,4352323.888911,40135346.208694,4151985.827131,311617.390000,3148559.281300,2135209.017758,2023-03-05,网+出
8,1,2758294.000000,1864926.000000,500518.000000,392846.000000,1436747.000000,1103512.000000,213701.000000,119534.000000,1191884.000000,924922.000000,176469.000000,90493.000000,44220519.670000,34475168.720000,6372451.140000,3372899.810000,32969054.209191,4156986.764809,311617.390000,3141605.511300,2134054.668723,2023-03-05,网约车
9,42,41637.000000,18455.000000,10176.000000,13006.000000,293.000000,159.000000,64.000000,70.000000,129.000000,90.000000,20.000000,19.000000,5746.680000,4380.760000,820.100000,545.820000,4469.160000,964.395005,19.560000,90.640000,27.200000,2023-03-05,专车


In [41]:
# 城市供需表
df = spark.sql('''
with t as (
    select
        city_id,
        sum(total_tsh) as total_tsh,
        sum(th_yj_tsh) as th_yj_tsh,
        sum(answer_order_cnt) as answer_order_cnt,
        sum(charge_dur) as charge_dur,
        dt,
        case
            when product_name = '特惠自营' then '特惠快车'
            when product_name = '泛快含优含拼（快车（普通快车、A+、车大、巡游网约车）、市内拼车、优享、D1、特惠快车、涅槃）' then '泛快大盘'
            else product_name
        end as product_name
    from
        mp_data.dm_trip_mp_sd_core_1d
    where
        dt = '2023-04-01'
        and product_name in (
            '特惠自营',
            '快车',
            '普通快车',
            '市内拼车',
            '专车',
            '出租车',
            '网约车',
            '网+出',
            '泛快含优含拼（快车（普通快车、A+、车大、巡游网约车）、市内拼车、优享、D1、特惠快车、涅槃）'
        )
        and city_id in (1, 42, 366)
    group by
        dt,
        city_id,
        product_name
)
select
    aa.city_id as city_id,
    total_tsh,
    --TSH（没有普通快车、市内拼车，使用快车替代普通快车）
    th_yj_tsh,
    --愿接特惠时长
    answer_order_cnt,
    --应答数
    charge_dur,
    --计费时长（没有普通快车、市内拼车、特惠快车，使用快车替代普通快车和特惠快车）
    --TPH = 完单数 / TSH
    --供需比 = TSH / 发单数
    --需供比 = 发单数 / TSH
    --应答率 = 应答数 / 发单数
    aa.dt as dt,
    aa.product_name as product_name
from
    (
        select
            a.dt as dt,
            a.city_id as city_id,
            a.product_name as product_name,
            total_tsh,
            th_yj_tsh,
            answer_order_cnt,
            case
                when a.product_name in ('特惠快车', '普通快车') then '快车'
                else a.product_name
            end as src_product_name
        from
            (
                select
                    dt,
                    city_id,
                    product_name,
                    th_yj_tsh,
                    answer_order_cnt
                from
                    t
                where
                    not product_name = '快车'
            ) a
            left join (
                select
                    dt,
                    city_id,
                    case
                        when product_name = '快车' then '普通快车'
                        else product_name
                    end as product_name,
                    total_tsh
                from
                    t
                where
                    not product_name = '普通快车'
            ) b on a.dt = b.dt
            and a.city_id = b.city_id
            and a.product_name = b.product_name
    ) aa
    left join (
        select
            dt,
            city_id,
            product_name,
            charge_dur
        from
            t
    ) bb on aa.dt = bb.dt
    and aa.city_id = bb.city_id
    and aa.src_product_name = bb.product_name
order by
    aa.city_id, aa.product_name
limit
    100
''').toPandas()
df

,city_id,total_tsh,th_yj_tsh,answer_order_cnt,charge_dur,dt,product_name
0,1,97288.833333,0.000000,109434,40599.469444,2023-03-05,专车
1,1,169925.000000,0.000000,200841,76085.962778,2023-03-05,出租车
2,1,0.000000,0.000000,123379,0.000000,2023-03-05,市内拼车
3,1,569615.283333,0.000000,737506,312652.626389,2023-03-05,普通快车
4,1,683174.033333,0.000000,1226477,361451.639167,2023-03-05,泛快大盘
5,1,220070.900000,188865.633333,248616,312652.626389,2023-03-05,特惠快车
6,1,911462.033333,0.000000,1550619,477017.661111,2023-03-05,网+出
7,1,786044.716667,0.000000,1349778,403272.258333,2023-03-05,网约车
8,42,287.933333,0.000000,158,37.430556,2023-03-05,专车
9,42,11516.816667,0.000000,7543,1708.169167,2023-03-05,出租车


In [46]:
# 冒泡订单表
df = spark.sql('''
select
    city_id,
    count(
        if(
            bubble_hour >= 07
            and bubble_hour < 10,
            bubble_hour,
            null
        )
    ) as morning_peak_bubble_cnt,
    --早高峰冒泡数
    count(
        if(
            bubble_hour >= 17
            and bubble_hour < 20,
            bubble_hour,
            null
        )
    ) as evening_peak_bubble_cnt,
    --晚高峰冒泡数
    count(
        if(
            bubble_hour >= 21
            or bubble_hour < 5,
            bubble_hour,
            null
        )
    ) as night_peak_bubble_cnt, --夜高峰冒泡数
    dt,
    level_1_product,
    level_2_product,
    level_3_product
from
    (
        select
            dt,
            bubble_id,
            start_city_id as city_id,
            substring(bubble_time, 12, 2) as bubble_hour
        from
            dd_cdm.dwd_trd_bubble_distinct_di
        where
            dt = '2023-04-01'
            and level1_product_id in (110000, 120000)
            and start_city in (1, 42, 366)
    ) a
    left join(
        select
            distinct bubble_id,
            level_1_product,
            level_2_product,
            level_3_product
        from
            dd_cdm.dwd_trip_trd_bubble_anycar_di
        where
            dt = '2023-04-01'
            and level_1_product in (110000, 120000)
            and start_city in (1, 42, 366)
    ) b on a.bubble_id = b.bubble_id
group by
    dt,
    city_id,
    level_1_product,
    level_2_product,
    level_3_product
order by
    city_id, level_3_product
limit
    100
''').toPandas()
df

,city_id,morning_peak_bubble_cnt,evening_peak_bubble_cnt,night_peak_bubble_cnt,dt,level_1_product,level_2_product,level_3_product
0,1,8,246,331,2023-03-05,NaN,NaN,NaN
1,1,1111244,1639973,1702756,2023-03-05,110000.0,110100.0,110101.0
2,1,97354,158648,171160,2023-03-05,110000.0,110100.0,110102.0
3,1,984476,1426873,1463616,2023-03-05,110000.0,110900.0,110103.0
4,1,303067,443408,228771,2023-03-05,110000.0,110200.0,110201.0
5,1,355295,533666,281690,2023-03-05,110000.0,110200.0,110203.0
6,1,21,39,50,2023-03-05,110000.0,110200.0,110299.0
7,1,36336,47079,28145,2023-03-05,110000.0,110300.0,110302.0
8,1,32210,41523,24776,2023-03-05,110000.0,110300.0,110303.0
9,1,976496,1446926,1518204,2023-03-05,110000.0,110400.0,110401.0


In [ ]:
# 呼叫订单表
df = spark.sql('''
select
    city_id,
    count(
        if(
            call_hour >= 07
            and call_hour < 10,
            call_hour,
            null
        )
    ) as morning_peak_call_cnt,
    --早高峰发单数
    count(
        if(
            call_hour >= 17
            and call_hour < 20,
            call_hour,
            null
        )
    ) as evening_peak_call_cnt,
    --晚高峰发单数
    count(
        if(
            call_hour >= 21
            or call_hour < 5,
            call_hour,
            null
        )
    ) as night_peak_call_cnt,
    --夜高峰发单数
    sum(is_lineup) as lineup_call_cnt,
    --排队订单数
    sum(
        if(
            call_hour >= 07
            and call_hour < 10,
            is_lineup,
            0
        )
    ) as morning_peak_lineup_call_cnt,
    --早高峰排队订单数
    sum(
        if(
            call_hour >= 17
            and call_hour < 20,
            is_lineup,
            0
        )
    ) as evening_peak_lineup_call_cnt,
    --晚高峰排队订单数
    sum(
        if(
            call_hour >= 21
            or call_hour < 5,
            is_lineup,
            0
        )
    ) as night_peak_lineup_call_cnt,
    --夜高峰排队订单数
    sum(est_dur) as est_dur,
    --预估时长
    sum(est_arrive_dur) as est_arrive_dur,
    --预计接驾时长
    sum(
        if(
            call_hour >= 07
            and call_hour < 10,
            est_arrive_dur,
            0
        )
    ) as morning_peak_est_arrive_dur,
    --早高峰预计接驾时长
    sum(
        if(
            call_hour >= 17
            and call_hour < 20,
            est_arrive_dur,
            0
        )
    ) as evening_peak_est_arrive_dur,
    --晚高峰预计接驾时长
    sum(
        if(
            call_hour >= 21
            or call_hour < 5,
            est_arrive_dur,
            0
        )
    ) as night_peak_est_arrive_dur,
    --夜高峰预计接驾时长
    sum(est_arrive_dis) as est_arrive_dis,
    --预计接驾距离
    dt,
    level_1_product,
    level_2_product,
    level_3_product
from
    (
        select
            dt,
            call_city as city_id,
            level_1_product,
            level_2_product,
            level_3_product,
            substring(call_time, 12, 2) as call_hour,
            is_lineup,
            est_dur,
            est_arrive_dur,
            est_arrive_dis
        from
            decdm.dwd_gflt_ord_order_base_di
        where
            dt = '2023-04-01'
            and level_1_product in (110000, 120000)
            and call_city in (1, 42, 366)
    ) t
group by
    dt,
    city_id,
    level_1_product,
    level_2_product,
    level_3_product
order by
    city_id, level_3_product
limit
    100
''').toPandas()
df

,city_id,morning_peak_call_cnt,evening_peak_call_cnt,night_peak_call_cnt,lineup_call_cnt,morning_peak_lineup_call_cnt,evening_peak_lineup_call_cnt,night_peak_lineup_call_cnt,est_dur,est_arrive_dur,morning_peak_est_arrive_dur,evening_peak_est_arrive_dur,night_peak_est_arrive_dur,est_arrive_dis,dt,level_1_product,level_2_product,level_3_product
0,1,96843,147469,162499,53368,1430,12594,19527,909308700.000000,146582578.000000,16443250.000000,26802515.000000,37607382.000000,934298215.000000,2023-03-05,110000,110100,110101
1,1,2760,5166,5863,34,1,0,30,31154580.000000,4421721.000000,506722.000000,661011.000000,1372135.000000,24643405.000000,2023-03-05,110000,110100,110102
2,1,31940,48564,45254,8,0,1,5,313958700.000000,62217232.000000,7702715.000000,11364230.000000,12443259.000000,372230728.000000,2023-03-05,110000,110900,110103
3,1,14030,19810,10418,954,69,243,261,147423120.000000,21399328.000000,3050482.000000,4704202.000000,2176815.000000,125187335.000000,2023-03-05,110000,110200,110201
4,1,10923,13922,7350,4696,213,1513,1262,68139900.000000,12926570.000000,2399391.000000,3033197.000000,1644188.000000,73200144.000000,2023-03-05,110000,110200,110203
5,1,1514,1942,871,0,0,0,0,33291720.000000,3928889.000000,590993.000000,891778.000000,298015.000000,26454333.000000,2023-03-05,110000,110300,110302
6,1,593,669,581,325,9,75,128,12375120.000000,1228753.000000,206397.000000,221760.000000,193974.000000,7595376.000000,2023-03-05,110000,110300,110303
7,1,10497,18927,19096,2877,33,317,2092,127721940.000000,21775359.000000,2064859.000000,4238033.000000,5444366.000000,137813734.000000,2023-03-05,110000,110400,110401
8,1,9540,19415,24429,2178,7,140,1839,148434300.000000,30525616.000000,2704240.000000,5748821.000000,7944831.000000,209748904.000000,2023-03-05,110000,110500,110501
9,1,949,1976,2394,32,0,1,30,14729700.000000,4249941.000000,442697.000000,787965.000000,882312.000000,30856417.000000,2023-03-05,110000,110500,110502


In [52]:
# 完单订单表
df = spark.sql('''
select
    city_id,
    count(
        if(
            end_charge_hour >= 07
            and end_charge_hour < 10,
            end_charge_hour,
            null
        )
    ) as morning_peak_success_cnt,
    --早高峰完单数
    count(
        if(
            end_charge_hour >= 17
            and end_charge_hour < 20,
            end_charge_hour,
            null
        )
    ) as evening_peak_success_cnt,
    --晚高峰完单数
    count(
        if(
            end_charge_hour >= 21
            or end_charge_hour < 5,
            end_charge_hour,
            null
        )
    ) as night_peak_success_cnt,
    --夜高峰完单数
    sum(
        if(
            end_charge_hour >= 07
            and end_charge_hour < 10,
            gmv_amt / 100,
            0
        )
    ) as morning_peak_gmv,
    --早高峰完单GMV
    sum(
        if(
            end_charge_hour >= 17
            and end_charge_hour < 20,
            gmv_amt,
            0
        )
    ) as evening_peak_gmv,
    --晚高峰完单GMV
    sum(
        if(
            end_charge_hour >= 21
            or end_charge_hour < 5,
            gmv_amt,
            0
        )
    ) as night_peak_gmv,
    --夜高峰完单GMV
    count(
        if(
            est_dis >= 15000,
            est_dis,
            null
        )
    ) as long_dis_success_cnt,
    --长距离完单数
    count(
        if(
            est_dis < 15000,
            est_dis,
            null
        )
    ) as short_dis_success_cnt,
    --短距离完单数
    sum(
        if(
            est_dis >= 15000,
            gmv_amt / 100,
            0
        )
    ) as long_dis_gmv,
    --长距离完单GMV
    sum(
        if(
            est_dis < 15000,
            gmv_amt / 100,
            0
        )
    ) as short_dis_gmv, --短距离完单GMV
    dt,
    level_1_product,
    level_2_product,
    level_3_product
from
    (
        select
            dt,
            city_id,
            level_1_product,
            level_2_product,
            level_3_product,
            substring(end_charge_time, 12, 2) as end_charge_hour,
            est_dis,
            gmv_amt
        from
            dd_cdm.dwd_trip_trd_attribute_call_order_base_di
        where
            dt = '2023-04-01'
            and level_1_product in (110000, 120000)
            and city_id in (1, 42, 366)
    ) t
group by
    dt,
    city_id,
    level_1_product,
    level_2_product,
    level_3_product
order by
    city_id, level_3_product
limit
    100
''').toPandas()
df

,city_id,morning_peak_success_cnt,evening_peak_success_cnt,night_peak_success_cnt,morning_peak_gmv,evening_peak_gmv,night_peak_gmv,long_dis_success_cnt,short_dis_success_cnt,long_dis_gmv,short_dis_gmv,dt,level_1_product,level_2_product,level_3_product
0,1,80769,136384,218704,2477157.2300000000,388356376.000000,509255873.000000,151550,629622,9990307.7000000000,11788318.4500000000,2023-03-05,110000,110100,110101
1,1,2241,5088,7211,66541.5600000000,15324553.000000,20804015.000000,4898,21352,326815.6800000000,461457.6200000000,2023-03-05,110000,110100,110102
2,1,25884,46289,64516,804490.6400000000,147464583.000000,175154305.000000,59287,194530,4107630.9700000000,3780143.1700000000,2023-03-05,110000,110900,110103
3,1,7604,13402,42901,261027.4400000000,51408072.000000,38995028.000000,29010,65228,1250688.6200000000,1094229.4000000000,2023-03-05,110000,110200,110201
4,1,8796,12701,13645,223051.9900000000,33511507.000000,24880409.000000,8631,50165,472054.5100000000,985663.2200000000,2023-03-05,110000,110200,110203
5,1,689,1433,3956,43980.7700000000,9763627.000000,7616356.000000,9304,0,442393.0100000000,0E-10,2023-03-05,110000,110300,110302
6,1,382,607,1209,28878.6500000000,5362960.000000,6652403.000000,3618,0,273907.4000000000,0E-10,2023-03-05,110000,110300,110303
7,1,8496,17576,26507,354388.5300000000,66695730.000000,75214187.000000,24111,72855,1833315.6500000000,1672434.4000000000,2023-03-05,110000,110400,110401
8,1,7680,18622,32449,532963.1300000000,117072900.000000,149630521.000000,30655,71651,3348163.0800000000,2566180.2800000000,2023-03-05,110000,110500,110501
9,1,606,1669,3942,71900.7000000000,16191306.000000,19239951.000000,2642,7907,468268.7200000000,395293.1200000000,2023-03-05,110000,110500,110502


In [55]:
# 折扣分布表
df = spark.sql('''
with t as(
    select
        area_id as city_id,
        count(if(exp_discount < 70, exp_discount, null)) as discount_00_to_70_cnt,
        count(
            if(
                exp_discount >= 70
                and exp_discount < 80,
                exp_discount,
                null
            )
        ) as discount_70_to_80_cnt,
        count(
            if(
                exp_discount >= 80
                and exp_discount < 90,
                exp_discount,
                null
            )
        ) as discount_80_to_90_cnt,
        count(
            if(
                exp_discount >= 90
                and exp_discount < 95,
                exp_discount,
                null
            )
        ) as discount_90_to_95_cnt,
        count(
            if(
                exp_discount >= 95
                and exp_discount <= 100,
                exp_discount,
                null
            )
        ) as discount_95_to_100_cnt,
        dt,
        current_product_type
    from
        mp_data.dwd_trip_mp_hufan_bubble_di
    where
        dt = '2023-04-01'
        and current_product_type in (
            206,
            2061,
            210,
            208,
            315,
            209,
            100,
            400,
            900,
            314,
            2200,
            500
        )
        and area_id in (1, 42, 366)
    group by
        dt,
        area_id,
        current_product_type
)
select
    city_id,
    dt,
    case
        when current_product_type = 314 then '特惠快车'
        when current_product_type = 206 then '普通快车'
        when current_product_type in (100, 400) then '专车'
        when current_product_type = 500 then '出租车'
    end as product_name,
    sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
    sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
    sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
    sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
    sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
from
    t
where
    current_product_type in (314, 206, 100, 400, 500)
group by
    dt,
    city_id,
    case
        when current_product_type = 314 then '特惠快车'
        when current_product_type = 206 then '普通快车'
        when current_product_type in (100, 400) then '专车'
        when current_product_type = 500 then '出租车'
    end
union
select
    city_id,
    dt,
    '网约车' product_name,
    sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
    sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
    sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
    sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
    sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
from
    t
where
    current_product_type in (
        206,
        2061,
        210,
        208,
        315,
        209,
        100,
        400,
        900,
        314,
        2200
    )
group by
    dt,
    city_id
union
select
    city_id,
    dt,
    '网+出' product_name,
    sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
    sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
    sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
    sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
    sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
from
    t
where
    current_product_type in (
        206,
        2061,
        210,
        208,
        315,
        209,
        100,
        400,
        900,
        314,
        2200,
        500
    )
group by
    dt,
    city_id
union
select
    city_id,
    dt,
    '新泛快' product_name,
    sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
    sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
    sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
    sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
    sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
from
    t
where
    current_product_type in (206, 2061, 314)
group by
    dt,
    city_id
union
select
    city_id,
    dt,
    '泛快大盘' product_name,
    sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
    sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
    sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
    sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
    sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
from
    t
where
    current_product_type in (206, 2061, 210, 208, 315, 209, 314)
group by
    dt,
    city_id;
group by
    dt,
    city_id
order by
    city_id, product_name
limit
    100
''').toPandas()
df

,city_id,discount_00_to_70_cnt,discount_70_to_80_cnt,discount_80_to_90_cnt,discount_90_to_95_cnt,discount_95_to_100_cnt,dt,product_name
0,1,40826,107377,1315500,26227,9104365,2023-03-05,专车
1,1,0,0,0,0,0,2023-03-05,出租车
2,1,19976,64712,182608,999861,6676941,2023-03-05,新泛快
3,1,0,7148,13119,4892,3931599,2023-03-05,普通快车
4,1,19976,64712,182608,999861,6676943,2023-03-05,泛快大盘
5,1,19976,57564,169489,994969,2745342,2023-03-05,特惠快车
6,1,60802,814903,2467032,1768694,16361716,2023-03-05,网+出
7,1,60802,814903,2467032,1768694,16361716,2023-03-05,网约车
8,10,407787,64551,728301,8796,2672849,2023-03-05,专车
9,10,0,0,0,0,0,2023-03-05,出租车


In [5]:
# 订单合并表（不含映射）
df = spark.sql('''
select
    aa.city_id as city_id,
    morning_peak_bubble_cnt,
    evening_peak_bubble_cnt,
    night_peak_bubble_cnt,
    morning_peak_call_cnt,
    evening_peak_call_cnt,
    night_peak_call_cnt,
    lineup_call_cnt,
    morning_peak_lineup_call_cnt,
    evening_peak_lineup_call_cnt,
    night_peak_lineup_call_cnt,
    est_dur,
    est_arrive_dur,
    morning_peak_est_arrive_dur,
    evening_peak_est_arrive_dur,
    night_peak_est_arrive_dur,
    est_arrive_dis,
    morning_peak_success_cnt,
    evening_peak_success_cnt,
    night_peak_success_cnt,
    morning_peak_gmv,
    evening_peak_gmv,
    night_peak_gmv,
    long_dis_success_cnt,
    short_dis_success_cnt,
    long_dis_gmv,
    short_dis_gmv,
    aa.dt as dt,
    aa.level_1_product as level_1_product,
    aa.level_2_product as level_2_product,
    aa.level_3_product as level_3_product
from
    (
        select
            a.city_id as city_id,
            morning_peak_bubble_cnt,
            evening_peak_bubble_cnt,
            night_peak_bubble_cnt,
            morning_peak_call_cnt,
            evening_peak_call_cnt,
            night_peak_call_cnt,
            lineup_call_cnt,
            morning_peak_lineup_call_cnt,
            evening_peak_lineup_call_cnt,
            night_peak_lineup_call_cnt,
            est_dur,
            est_arrive_dur,
            morning_peak_est_arrive_dur,
            evening_peak_est_arrive_dur,
            night_peak_est_arrive_dur,
            est_arrive_dis,
            a.dt as dt,
            a.level_1_product as level_1_product,
            a.level_2_product as level_2_product,
            a.level_3_product as level_3_product
        from
            (
                select
                    city_id,
                    count(
                        if(
                            bubble_hour >= 07
                            and bubble_hour < 10,
                            bubble_hour,
                            null
                        )
                    ) as morning_peak_bubble_cnt,
                    --早高峰冒泡数
                    count(
                        if(
                            bubble_hour >= 17
                            and bubble_hour < 20,
                            bubble_hour,
                            null
                        )
                    ) as evening_peak_bubble_cnt,
                    --晚高峰冒泡数
                    count(
                        if(
                            bubble_hour >= 21
                            or bubble_hour < 5,
                            bubble_hour,
                            null
                        )
                    ) as night_peak_bubble_cnt,
                    --夜高峰冒泡数
                    dt,
                    level_1_product,
                    level_2_product,
                    level_3_product
                from
                    (
                        select
                            dt,
                            bubble_id,
                            start_city_id as city_id,
                            substring(bubble_time, 12, 2) as bubble_hour
                        from
                            dd_cdm.dwd_trd_bubble_distinct_di
                        where
                            dt = '2023-04-01'
                            and level1_product_id in (110000, 120000)
                            and start_city_id in (1, 42, 366)
                    ) a
                    left join(
                        select
                            distinct bubble_id,
                            level_1_product,
                            level_2_product,
                            level_3_product
                        from
                            dd_cdm.dwd_trip_trd_bubble_anycar_di
                        where
                            dt = '2023-04-01'
                            and level_1_product in (110000, 120000)
                            and start_city_id in (1, 42, 366)
                    ) b on a.bubble_id = b.bubble_id
                group by
                    dt,
                    city_id,
                    level_1_product,
                    level_2_product,
                    level_3_product
            ) a full
            join (
                select
                    city_id,
                    count(
                        if(
                            call_hour >= 07
                            and call_hour < 10,
                            call_hour,
                            null
                        )
                    ) as morning_peak_call_cnt,
                    --早高峰发单数
                    count(
                        if(
                            call_hour >= 17
                            and call_hour < 20,
                            call_hour,
                            null
                        )
                    ) as evening_peak_call_cnt,
                    --晚高峰发单数
                    count(
                        if(
                            call_hour >= 21
                            or call_hour < 5,
                            call_hour,
                            null
                        )
                    ) as night_peak_call_cnt,
                    --夜高峰发单数
                    sum(is_lineup) as lineup_call_cnt,
                    --排队订单数
                    sum(
                        if(
                            call_hour >= 07
                            and call_hour < 10,
                            is_lineup,
                            0
                        )
                    ) as morning_peak_lineup_call_cnt,
                    --早高峰排队订单数
                    sum(
                        if(
                            call_hour >= 17
                            and call_hour < 20,
                            is_lineup,
                            0
                        )
                    ) as evening_peak_lineup_call_cnt,
                    --晚高峰排队订单数
                    sum(
                        if(
                            call_hour >= 21
                            or call_hour < 5,
                            is_lineup,
                            0
                        )
                    ) as night_peak_lineup_call_cnt,
                    --夜高峰排队订单数
                    sum(est_dur) as est_dur,
                    --预估时长
                    sum(est_arrive_dur) as est_arrive_dur,
                    --预计接驾时长
                    sum(
                        if(
                            call_hour >= 07
                            and call_hour < 10,
                            est_arrive_dur,
                            0
                        )
                    ) as morning_peak_est_arrive_dur,
                    --早高峰预计接驾时长
                    sum(
                        if(
                            call_hour >= 17
                            and call_hour < 20,
                            est_arrive_dur,
                            0
                        )
                    ) as evening_peak_est_arrive_dur,
                    --晚高峰预计接驾时长
                    sum(
                        if(
                            call_hour >= 21
                            or call_hour < 5,
                            est_arrive_dur,
                            0
                        )
                    ) as night_peak_est_arrive_dur,
                    --夜高峰预计接驾时长
                    sum(est_arrive_dis) as est_arrive_dis,
                    --预计接驾距离
                    dt,
                    level_1_product,
                    level_2_product,
                    level_3_product
                from
                    (
                        select
                            dt,
                            call_city as city_id,
                            level_1_product,
                            level_2_product,
                            level_3_product,
                            substring(call_time, 12, 2) as call_hour,
                            is_lineup,
                            est_dur,
                            est_arrive_dur,
                            est_arrive_dis
                        from
                            decdm.dwd_gflt_ord_order_base_di
                        where
                            dt = '2023-04-01'
                            and level_1_product in (110000, 120000)
                            and call_city in (1, 42, 366)
                    ) t
                group by
                    dt,
                    city_id,
                    level_1_product,
                    level_2_product,
                    level_3_product
            ) b on a.dt = b.dt
            and a.city_id = b.city_id
            and a.level_1_product = b.level_1_product
            and a.level_2_product = b.level_2_product
            and a.level_3_product = b.level_3_product
    ) aa full
    join (
        select
            city_id,
            count(
                if(
                    end_charge_hour >= 07
                    and end_charge_hour < 10,
                    end_charge_hour,
                    null
                )
            ) as morning_peak_success_cnt,
            --早高峰完单数
            count(
                if(
                    end_charge_hour >= 17
                    and end_charge_hour < 20,
                    end_charge_hour,
                    null
                )
            ) as evening_peak_success_cnt,
            --晚高峰完单数
            count(
                if(
                    end_charge_hour >= 21
                    or end_charge_hour < 5,
                    end_charge_hour,
                    null
                )
            ) as night_peak_success_cnt,
            --夜高峰完单数
            sum(
                if(
                    end_charge_hour >= 07
                    and end_charge_hour < 10,
                    gmv_amt / 100,
                    0
                )
            ) as morning_peak_gmv,
            --早高峰完单GMV
            sum(
                if(
                    end_charge_hour >= 17
                    and end_charge_hour < 20,
                    gmv_amt,
                    0
                )
            ) as evening_peak_gmv,
            --晚高峰完单GMV
            sum(
                if(
                    end_charge_hour >= 21
                    or end_charge_hour < 5,
                    gmv_amt,
                    0
                )
            ) as night_peak_gmv,
            --夜高峰完单GMV
            count(
                if(
                    est_dis >= 15000,
                    est_dis,
                    null
                )
            ) as long_dis_success_cnt,
            --长距离完单数
            count(
                if(
                    est_dis < 15000,
                    est_dis,
                    null
                )
            ) as short_dis_success_cnt,
            --短距离完单数
            sum(
                if(
                    est_dis >= 15000,
                    gmv_amt / 100,
                    0
                )
            ) as long_dis_gmv,
            --长距离完单GMV
            sum(
                if(
                    est_dis < 15000,
                    gmv_amt / 100,
                    0
                )
            ) as short_dis_gmv,
            --短距离完单GMV
            dt,
            level_1_product,
            level_2_product,
            level_3_product
        from
            (
                select
                    dt,
                    city_id,
                    level_1_product,
                    level_2_product,
                    level_3_product,
                    substring(end_charge_time, 12, 2) as end_charge_hour,
                    est_dis,
                    gmv_amt
                from
                    dd_cdm.dwd_trip_trd_attribute_call_order_base_di
                where
                    dt = '2023-04-01'
                    and level_1_product in (110000, 120000)
                    and city_id in (1, 42, 366)
            ) t
        group by
            dt,
            city_id,
            level_1_product,
            level_2_product,
            level_3_product
    ) bb on aa.dt = bb.dt
    and aa.city_id = bb.city_id
    and aa.level_1_product = bb.level_1_product
    and aa.level_2_product = bb.level_2_product
    and aa.level_3_product = bb.level_3_product
order by
    aa.city_id, aa.level_1_product, aa.level_2_product, aa.level_3_product
''').toPandas()
df

,city_id,morning_peak_bubble_cnt,evening_peak_bubble_cnt,night_peak_bubble_cnt,morning_peak_call_cnt,evening_peak_call_cnt,night_peak_call_cnt,lineup_call_cnt,morning_peak_lineup_call_cnt,evening_peak_lineup_call_cnt,night_peak_lineup_call_cnt,est_dur,est_arrive_dur,morning_peak_est_arrive_dur,evening_peak_est_arrive_dur,night_peak_est_arrive_dur,est_arrive_dis,morning_peak_success_cnt,evening_peak_success_cnt,night_peak_success_cnt,morning_peak_gmv,evening_peak_gmv,night_peak_gmv,long_dis_success_cnt,short_dis_success_cnt,long_dis_gmv,short_dis_gmv,dt,level_1_product,level_2_product,level_3_product
0,1,49,402,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,2023-04-01,NaN,NaN,NaN
1,1,2470041,2278228,2593748,133613.0,151993.0,206537.0,229465.0,34640.0,28370.0,37963.0,1194805140.000000,238604949.000000,35518977.000000,33646279.000000,48522001.000000,1494964495.000000,88697.0,137824.0,344190.0,2704411.8000000000,401362106.000000,588125364.000000,185399.0,755262.0,11203996.2300000000,12778484.6200000000,2023-04-01,110000.0,110100.0,110101.0
2,1,619286,667631,723728,18193.0,21233.0,27925.0,2421.0,564.0,145.0,875.0,158878440.000000,34661202.000000,5888610.000000,4568152.000000,6969150.000000,204802745.000000,13314.0,20209.0,39305.0,420855.5900000000,63783988.000000,93343898.000000,23430.0,101750.0,1636423.5700000000,2120870.0900000000,2023-04-01,110000.0,110100.0,110102.0
3,1,449364,509889,270952,16130.0,19222.0,9915.0,6732.0,1562.0,932.0,456.0,143316660.000000,18119506.000000,3243037.000000,4083897.000000,1704569.000000,106134692.000000,6781.0,11464.0,47840.0,225742.5500000000,41792249.000000,29182857.000000,30053.0,60306.0,958626.1900000000,895591.6900000000,2023-04-01,110000.0,110200.0,110201.0
4,1,570279,621217,327374,24524.0,22809.0,12993.0,44592.0,10880.0,5522.0,3199.0,143848680.000000,30770339.000000,6962387.000000,5320296.000000,2956611.000000,178066242.000000,17924.0,20959.0,28287.0,470224.9000000000,58930977.000000,44394958.000000,20126.0,98747.0,1095663.4600000000,1906142.3900000000,2023-04-01,110000.0,110200.0,110203.0
5,1,66,49,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,2023-04-01,110000.0,110200.0,110299.0
6,1,53010,46150,29093,1896.0,1645.0,1035.0,0.0,0.0,0.0,0.0,36440820.000000,4061557.000000,818653.000000,710426.000000,314871.000000,27237034.000000,669.0,1227.0,4526.0,42690.6900000000,8549852.000000,6403613.000000,9581.0,0.0,423873.2600000000,0E-10,2023-04-01,110000.0,110300.0,110302.0
7,1,49870,43494,26792,803.0,665.0,631.0,1148.0,249.0,111.0,139.0,16051020.000000,1661044.000000,325517.000000,225409.000000,207808.000000,10371127.000000,426.0,666.0,1446.0,33159.0600000000,5776317.000000,6367359.000000,4282.0,0.0,313859.1000000000,0E-10,2023-04-01,110000.0,110300.0,110303.0
8,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None,2023-04-01,110000.0,110300.0,110304.0
9,1,2261002,2021313,2306316,23237.0,26173.0,31041.0,26328.0,4579.0,2358.0,6391.0,226495020.000000,51402912.000000,8000399.000000,7054041.000000,9187120.000000,301497289.000000,15726.0,24426.0,50597.0,575999.9500000000,88572717.000000,108418089.000000,34102.0,129654.0,2461734.9900000000,2731955.3300000000,2023-04-01,110000.0,110400.0,110401.0


In [13]:
# 订单合并表（含映射）
df = spark.sql('''
with temp as(
    select
        aa.city_id as city_id,
        morning_peak_bubble_cnt,
        evening_peak_bubble_cnt,
        night_peak_bubble_cnt,
        morning_peak_call_cnt,
        evening_peak_call_cnt,
        night_peak_call_cnt,
        lineup_call_cnt,
        morning_peak_lineup_call_cnt,
        evening_peak_lineup_call_cnt,
        night_peak_lineup_call_cnt,
        est_dur,
        est_arrive_dur,
        morning_peak_est_arrive_dur,
        evening_peak_est_arrive_dur,
        night_peak_est_arrive_dur,
        est_arrive_dis,
        morning_peak_success_cnt,
        evening_peak_success_cnt,
        night_peak_success_cnt,
        morning_peak_gmv,
        evening_peak_gmv,
        night_peak_gmv,
        long_dis_success_cnt,
        short_dis_success_cnt,
        long_dis_gmv,
        short_dis_gmv,
        aa.dt as dt,
        aa.level_1_product as level_1_product,
        aa.level_2_product as level_2_product,
        aa.level_3_product as level_3_product
    from
        (
            select
                a.city_id as city_id,
                morning_peak_bubble_cnt,
                evening_peak_bubble_cnt,
                night_peak_bubble_cnt,
                morning_peak_call_cnt,
                evening_peak_call_cnt,
                night_peak_call_cnt,
                lineup_call_cnt,
                morning_peak_lineup_call_cnt,
                evening_peak_lineup_call_cnt,
                night_peak_lineup_call_cnt,
                est_dur,
                est_arrive_dur,
                morning_peak_est_arrive_dur,
                evening_peak_est_arrive_dur,
                night_peak_est_arrive_dur,
                est_arrive_dis,
                a.dt as dt,
                a.level_1_product as level_1_product,
                a.level_2_product as level_2_product,
                a.level_3_product as level_3_product
            from
                (
                    select
                        city_id,
                        count(
                            if(
                                bubble_hour >= 07
                                and bubble_hour < 10,
                                bubble_hour,
                                null
                            )
                        ) as morning_peak_bubble_cnt,
                        --早高峰冒泡数
                        count(
                            if(
                                bubble_hour >= 17
                                and bubble_hour < 20,
                                bubble_hour,
                                null
                            )
                        ) as evening_peak_bubble_cnt,
                        --晚高峰冒泡数
                        count(
                            if(
                                bubble_hour >= 21
                                or bubble_hour < 5,
                                bubble_hour,
                                null
                            )
                        ) as night_peak_bubble_cnt,
                        --夜高峰冒泡数
                        dt,
                        level_1_product,
                        level_2_product,
                        level_3_product
                    from
                        (
                            select
                                dt,
                                bubble_id,
                                start_city_id as city_id,
                                substring(bubble_time, 12, 2) as bubble_hour
                            from
                                dd_cdm.dwd_trd_bubble_distinct_di
                            where
                                dt = '2023-04-01'
                                and level1_product_id in (110000, 120000)
                                and start_city_id in (1, 42, 366)
                        ) a
                        left join(
                            select
                                distinct bubble_id,
                                level_1_product,
                                level_2_product,
                                level_3_product
                            from
                                dd_cdm.dwd_trip_trd_bubble_anycar_di
                            where
                                dt = '2023-04-01'
                                and level_1_product in (110000, 120000)
                                and start_city_id in (1, 42, 366)
                        ) b on a.bubble_id = b.bubble_id
                    group by
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product
                ) a full
                join (
                    select
                        city_id,
                        count(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                call_hour,
                                null
                            )
                        ) as morning_peak_call_cnt,
                        --早高峰发单数
                        count(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                call_hour,
                                null
                            )
                        ) as evening_peak_call_cnt,
                        --晚高峰发单数
                        count(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                call_hour,
                                null
                            )
                        ) as night_peak_call_cnt,
                        --夜高峰发单数
                        sum(is_lineup) as lineup_call_cnt,
                        --排队订单数
                        sum(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                is_lineup,
                                0
                            )
                        ) as morning_peak_lineup_call_cnt,
                        --早高峰排队订单数
                        sum(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                is_lineup,
                                0
                            )
                        ) as evening_peak_lineup_call_cnt,
                        --晚高峰排队订单数
                        sum(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                is_lineup,
                                0
                            )
                        ) as night_peak_lineup_call_cnt,
                        --夜高峰排队订单数
                        sum(est_dur) as est_dur,
                        --预估时长
                        sum(est_arrive_dur) as est_arrive_dur,
                        --预计接驾时长
                        sum(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                est_arrive_dur,
                                0
                            )
                        ) as morning_peak_est_arrive_dur,
                        --早高峰预计接驾时长
                        sum(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                est_arrive_dur,
                                0
                            )
                        ) as evening_peak_est_arrive_dur,
                        --晚高峰预计接驾时长
                        sum(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                est_arrive_dur,
                                0
                            )
                        ) as night_peak_est_arrive_dur,
                        --夜高峰预计接驾时长
                        sum(est_arrive_dis) as est_arrive_dis,
                        --预计接驾距离
                        dt,
                        level_1_product,
                        level_2_product,
                        level_3_product
                    from
                        (
                            select
                                dt,
                                call_city as city_id,
                                level_1_product,
                                level_2_product,
                                level_3_product,
                                substring(call_time, 12, 2) as call_hour,
                                is_lineup,
                                est_dur,
                                est_arrive_dur,
                                est_arrive_dis
                            from
                                decdm.dwd_gflt_ord_order_base_di
                            where
                                dt = '2023-04-01'
                                and level_1_product in (110000, 120000)
                                and call_city in (1, 42, 366)
                        ) t
                    group by
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product
                ) b on a.dt = b.dt
                and a.city_id = b.city_id
                and a.level_1_product = b.level_1_product
                and a.level_2_product = b.level_2_product
                and a.level_3_product = b.level_3_product
        ) aa full
        join (
            select
                city_id,
                count(
                    if(
                        end_charge_hour >= 07
                        and end_charge_hour < 10,
                        end_charge_hour,
                        null
                    )
                ) as morning_peak_success_cnt,
                --早高峰完单数
                count(
                    if(
                        end_charge_hour >= 17
                        and end_charge_hour < 20,
                        end_charge_hour,
                        null
                    )
                ) as evening_peak_success_cnt,
                --晚高峰完单数
                count(
                    if(
                        end_charge_hour >= 21
                        or end_charge_hour < 5,
                        end_charge_hour,
                        null
                    )
                ) as night_peak_success_cnt,
                --夜高峰完单数
                sum(
                    if(
                        end_charge_hour >= 07
                        and end_charge_hour < 10,
                        gmv_amt / 100,
                        0
                    )
                ) as morning_peak_gmv,
                --早高峰完单GMV
                sum(
                    if(
                        end_charge_hour >= 17
                        and end_charge_hour < 20,
                        gmv_amt,
                        0
                    )
                ) as evening_peak_gmv,
                --晚高峰完单GMV
                sum(
                    if(
                        end_charge_hour >= 21
                        or end_charge_hour < 5,
                        gmv_amt,
                        0
                    )
                ) as night_peak_gmv,
                --夜高峰完单GMV
                count(
                    if(
                        est_dis >= 15000,
                        est_dis,
                        null
                    )
                ) as long_dis_success_cnt,
                --长距离完单数
                count(
                    if(
                        est_dis < 15000,
                        est_dis,
                        null
                    )
                ) as short_dis_success_cnt,
                --短距离完单数
                sum(
                    if(
                        est_dis >= 15000,
                        gmv_amt / 100,
                        0
                    )
                ) as long_dis_gmv,
                --长距离完单GMV
                sum(
                    if(
                        est_dis < 15000,
                        gmv_amt / 100,
                        0
                    )
                ) as short_dis_gmv,
                --短距离完单GMV
                dt,
                level_1_product,
                level_2_product,
                level_3_product
            from
                (
                    select
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product,
                        substring(end_charge_time, 12, 2) as end_charge_hour,
                        est_dis,
                        gmv_amt
                    from
                        dd_cdm.dwd_trip_trd_attribute_call_order_base_di
                    where
                        dt = '2023-04-01'
                        and level_1_product in (110000, 120000)
                        and city_id in (1, 42, 366)
                ) t
            group by
                dt,
                city_id,
                level_1_product,
                level_2_product,
                level_3_product
        ) bb on aa.dt = bb.dt
        and aa.city_id = bb.city_id
        and aa.level_1_product = bb.level_1_product
        and aa.level_2_product = bb.level_2_product
        and aa.level_3_product = bb.level_3_product
)
select
    city_id,
    dt,
    case
        when level_3_product = 110103 then '特惠快车'
        when level_3_product = 110101 then '普通快车'
    end as product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_3_product in (110103, 110101)
group by
    city_id,
    dt,
    case
        when level_3_product = 110103 then '特惠快车'
        when level_3_product = 110101 then '普通快车'
    end
union
select
    city_id,
    dt,
    case
        when level_2_product = 110200 then '市内拼车'
        when level_2_product = 110500 then '专车'
    end as product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_2_product in (110200, 110500)
group by
    city_id,
    dt,
    case
        when level_2_product = 110200 then '市内拼车'
        when level_2_product = 110500 then '专车'
    end
union
select
    city_id,
    dt,
    case
        when level_1_product = 120000 then '出租车'
        when level_1_product = 110000 then '网约车'
    end as product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_1_product in (120000, 110000)
group by
    city_id,
    dt,
    case
        when level_1_product = 120000 then '出租车'
        when level_1_product = 110000 then '网约车'
    end
union
select
    city_id,
    dt,
    '网+出' product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_1_product in (120000, 110000)
group by
    city_id,
    dt
union
select
    city_id,
    dt,
    '新泛快' as product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_3_product in (110103, 110101, 110102)
group by
    city_id,
    dt
union
select
    city_id,
    dt,
    '泛快大盘' as product_name,
    sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
    sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
    sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
    sum(morning_peak_call_cnt) as morning_peak_call_cnt,
    sum(evening_peak_call_cnt) as evening_peak_call_cnt,
    sum(night_peak_call_cnt) as night_peak_call_cnt,
    sum(lineup_call_cnt) as lineup_call_cnt,
    sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
    sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
    sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
    sum(est_dur) as est_dur,
    sum(est_arrive_dur) as est_arrive_dur,
    sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
    sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
    sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
    sum(est_arrive_dis) as est_arrive_dis,
    sum(morning_peak_success_cnt) as morning_peak_success_cnt,
    sum(evening_peak_success_cnt) as evening_peak_success_cnt,
    sum(night_peak_success_cnt) as night_peak_success_cnt,
    sum(morning_peak_gmv) as morning_peak_gmv,
    sum(evening_peak_gmv) as evening_peak_gmv,
    sum(night_peak_gmv) as night_peak_gmv,
    sum(long_dis_success_cnt) as long_dis_success_cnt,
    sum(short_dis_success_cnt) as short_dis_success_cnt,
    sum(long_dis_gmv) as long_dis_gmv,
    sum(short_dis_gmv) as short_dis_gmv
from
    temp
where
    level_2_product in (110100, 110200, 110400, 110800, 110900)
group by
    city_id,
    dt
order by
   city_id, product_name
''').toPandas()
df

,city_id,dt,product_name,morning_peak_bubble_cnt,evening_peak_bubble_cnt,night_peak_bubble_cnt,morning_peak_call_cnt,evening_peak_call_cnt,night_peak_call_cnt,lineup_call_cnt,morning_peak_lineup_call_cnt,evening_peak_lineup_call_cnt,night_peak_lineup_call_cnt,est_dur,est_arrive_dur,morning_peak_est_arrive_dur,evening_peak_est_arrive_dur,night_peak_est_arrive_dur,est_arrive_dis,morning_peak_success_cnt,evening_peak_success_cnt,night_peak_success_cnt,morning_peak_gmv,evening_peak_gmv,night_peak_gmv,long_dis_success_cnt,short_dis_success_cnt,long_dis_gmv,short_dis_gmv
0,1,2023-04-01,专车,4598109,4135859,4671076,16239.0,25516.0,36216.0,8579.0,1456.0,661.0,3488.0,218777160.000000,59702098.000000,7615872.000000,10081336.000000,13853086.000000,388647574.000000,10783.0,23626.0,51765.0,733159.4800000000,153547731.000000,215246767.000000,39033.0,102420.0,4378904.7900000000,3751494.6300000000
1,1,2023-04-01,出租车,2351105,2157457,2454474,53604.0,40136.0,47993.0,13097.0,3180.0,883.0,2577.0,455420220.000000,73463389.000000,13491347.000000,10441279.000000,13237284.000000,455175394.000000,41058.0,35904.0,72352.0,1553877.3389710000,132935145.042900,174811951.829600,74960.0,198113.0,4917675.3675670000,4222397.9850870000
2,1,2023-04-01,市内拼车,1019709,1131155,598344,40654.0,42031.0,22908.0,51324.0,12442.0,6454.0,3655.0,287165340.000000,48889845.000000,10205424.000000,9404193.000000,4661180.000000,284200934.000000,24705.0,32423.0,76127.0,695967.4500000000,100723226.000000,73577815.000000,50179.0,159053.0,2054289.6500000000,2801734.0800000000
3,1,2023-04-01,新泛快,5405243,5046803,5686199,182937.0,219522.0,296573.0,231936.0,35212.0,28519.0,38850.0,1684554420.000000,349547843.000000,51707547.000000,51868310.000000,73802123.000000,2163419746.000000,125150.0,199468.0,469927.0,3882228.9600000000,602497835.000000,934378195.000000,274479.0,1034889.0,17392262.6000000000,18311860.9700000000
4,1,2023-04-01,普通快车,2470041,2278227,2593748,133613.0,151993.0,206537.0,229465.0,34640.0,28370.0,37963.0,1194805140.000000,238604949.000000,35518977.000000,33646279.000000,48522001.000000,1494964495.000000,88697.0,137824.0,344190.0,2704411.8000000000,401362106.000000,588125364.000000,185399.0,755262.0,11203996.2300000000,12778484.6200000000
5,1,2023-04-01,泛快大盘,8685955,8199276,8590867,246828.0,287726.0,350522.0,309588.0,52233.0,37331.0,48896.0,2198214780.000000,449840600.000000,69913370.000000,68326544.000000,87650423.000000,2749117969.000000,165581.0,256317.0,596651.0,5154196.3600000000,791793778.000000,1116374099.000000,358760.0,1323596.0,21908287.2400000000,23845550.3800000000
6,1,2023-04-01,特惠快车,2315916,2100945,2368723,31131.0,46296.0,62111.0,50.0,8.0,4.0,12.0,330870840.000000,76281692.000000,10299960.000000,13653879.000000,18310972.000000,463652506.000000,23139.0,41435.0,86432.0,756961.5700000000,137351741.000000,252908933.000000,65650.0,177877.0,4551842.8000000000,3412506.2600000000
7,1,2023-04-01,网+出,17999304,16613371,18091973,319906.0,356469.0,438703.0,333320.0,57198.0,39016.0,55733.0,2935006320.000000,592157186.000000,92535601.000000,90258599.000000,116382110.000000,3654849012.000000,218785.0,318378.0,729780.0,7564200.0289710000,1104295423.042900,1548429052.829600,488426.0,1628006.0,32312390.9075670000,32218462.4750870000
8,1,2023-04-01,网约车,15648199,14455914,15637499,266302.0,316333.0,390710.0,320223.0,54018.0,38133.0,53156.0,2479586100.000000,518693797.000000,79044254.000000,79817320.000000,103144826.000000,3199673618.000000,177727.0,282474.0,657428.0,6010322.6900000000,971360278.000000,1373617101.000000,413466.0,1429893.0,27394715.5400000000,27996064.4900000000
9,42,2023-04-01,专车,90708,139611,74015,33.0,51.0,6.0,1.0,0.0,0.0,0.0,237900.000000,60044.000000,6904.000000,19263.000000,2822.000000,498005.000000,20.0,41.0,46.0,1262.1800000000,147007.000000,56852.000000,54.0,119.0,4238.1000000000,2914.8700000000


In [4]:
# 最终合并表
df = spark.sql('''
with passenger_table as (
    select
        city_id,
        sum(bubble_cnt) as bubble_cnt,
        --冒泡数
        sum(if(pas_active_tag = 2, bubble_cnt, 0)) as high_freq_bubble_cnt,
        --中高活跃用户冒泡数
        sum(if(pas_active_tag = 1, bubble_cnt, 0)) as low_freq_bubble_cnt,
        --低活跃用户冒泡数
        sum(if(pas_active_tag = 0, bubble_cnt, 0)) as xcl_freq_bubble_cnt,
        --新沉流用户冒泡数
        sum(call_cnt) as call_cnt,
        --发单数
        sum(if(pas_active_tag = 2, call_cnt, 0)) as high_freq_call_cnt,
        --中高活跃用户发单数
        sum(if(pas_active_tag = 1, call_cnt, 0)) as low_freq_call_cnt,
        --低活跃用户发单数
        sum(if(pas_active_tag = 0, call_cnt, 0)) as xcl_freq_call_cnt,
        --新沉流用户发单数
        sum(success_cnt) as success_cnt,
        --完单数
        sum(if(pas_active_tag = 2, success_cnt, 0)) as high_freq_success_cnt,
        --中高活跃用户完单数
        sum(if(pas_active_tag = 1, success_cnt, 0)) as low_freq_success_cnt,
        --低活跃用户完单数
        sum(if(pas_active_tag = 0, success_cnt, 0)) as xcl_freq_success_cnt,
        --新沉流用户完单数
        sum(gmv) as gmv,
        --完单GMV
        sum(if(pas_active_tag = 2, gmv, 0)) as high_freq_gmv,
        --中高活跃用户完单GMV
        sum(if(pas_active_tag = 1, gmv, 0)) as low_freq_gmv,
        --低活跃用户完单GMV
        sum(if(pas_active_tag = 0, gmv, 0)) as xcl_freq_gmv,
        --新沉流用户完单GMV
        sum(driver_divide) as driver_divide,
        --司机抽成
        sum(business_profit) as business_profit,
        --毛利额
        sum(subsidy_hufan) as subsidy_hufan,
        --完单呼返
        sum(subsidy_c) as subsidy_c,
        --完单C补
        sum(subsidy_b) as subsidy_b,
        --完单B补
        dt,
        product_name
    from
        (
            select
                passenger_id,
                dt,
                city_id,
                case
                    when product_line = '特惠自营' then '特惠快车'
                    when product_line = '普通快车非拼' then '普通快车'
                    else product_line
                end as product_name,
                case
                    when product_line = '网+出' then '网+出'
                    when product_line = '出租车' then '网+出'
                    else '网约车'
                end as pas_product_name,
                sum(bubble_cnt) as bubble_cnt,
                --冒泡数
                sum(objective_call_cnt) as call_cnt,
                --发单数
                sum(suc_cnt) as success_cnt,
                --完单数
                sum(gmv) as gmv,
                --完单GMV
                sum(online_driver_divide) as driver_divide,
                --司机抽成
                sum(business_profit) as business_profit,
                --毛利额
                sum(subsidy_hufan) as subsidy_hufan,
                --完单呼返
                sum(subsidy_c) as subsidy_c,
                --完单C补
                sum(subsidy_b) as subsidy_b --完单B补
            from
                mp_data.dwm_trip_trd_psg_core_di
            where
                dt = '2023-04-01'
                and product_line in (
                    '特惠自营',
                    '普通快车非拼',
                    '市内拼车',
                    '专车',
                    '出租车',
                    '网约车',
                    '网+出',
                    '新泛快',
                    '泛快大盘'
                )
                and city_id in (1, 42, 366)
            group by
                passenger_id,
                dt,
                city_id,
                product_line
        ) a
        left join (
            select
                passenger_id,
                product_line as pas_product_name,
                case
                    when pas_call_active_tag in (3, 4) then 2 --中高活跃用户
                    when pas_call_active_tag = 2 then 1 --低活跃用户
                    else 0 --新沉流用户
                end as pas_active_tag
            from
                dd_cdm.dm_trip_pas_label_active_df
            where
                dt = '2023-04-01'
                and product_line in ('网约车', '网+出')
                and city_id in (1, 42, 366)
        ) b on a.passenger_id = b.passenger_id
        and a.pas_product_name = b.pas_product_name
    group by
        dt,
        city_id,
        product_name
),
t1 as (
    select
        city_id,
        sum(total_tsh) as total_tsh,
        sum(th_yj_tsh) as th_yj_tsh,
        sum(answer_order_cnt) as answer_order_cnt,
        sum(charge_dur) as charge_dur,
        dt,
        case
            when product_name = '特惠自营' then '特惠快车'
            when product_name = '泛快含优含拼（快车（普通快车、A+、车大、巡游网约车）、市内拼车、优享、D1、特惠快车、涅槃）' then '泛快大盘'
            else product_name
        end as product_name
    from
        mp_data.dm_trip_mp_sd_core_1d
    where
        dt = '2023-04-01'
        and product_name in (
            '特惠自营',
            '快车',
            '普通快车',
            '市内拼车',
            '专车',
            '出租车',
            '网约车',
            '网+出',
            '泛快含优含拼（快车（普通快车、A+、车大、巡游网约车）、市内拼车、优享、D1、特惠快车、涅槃）'
        )
        and city_id in (1, 42, 366)
    group by
        dt,
        city_id,
        product_name
),
city_table as (
    select
        aa.city_id as city_id,
        total_tsh,
        --TSH（没有普通快车、市内拼车，使用快车替代普通快车）
        th_yj_tsh,
        --愿接特惠时长
        answer_order_cnt,
        --应答数
        charge_dur,
        --计费时长（没有普通快车、市内拼车、特惠快车，使用快车替代普通快车和特惠快车）
        --TPH = 完单数 / TSH
        --供需比 = TSH / 发单数
        --需供比 = 发单数 / TSH
        --应答率 = 应答数 / 发单数
        aa.dt as dt,
        aa.product_name as product_name
    from
        (
            select
                a.dt as dt,
                a.city_id as city_id,
                a.product_name as product_name,
                total_tsh,
                th_yj_tsh,
                answer_order_cnt,
                case
                    when a.product_name in ('特惠快车', '普通快车') then '快车'
                    else a.product_name
                end as src_product_name
            from
                (
                    select
                        dt,
                        city_id,
                        product_name,
                        th_yj_tsh,
                        answer_order_cnt
                    from
                        t1
                    where
                        not product_name = '快车'
                ) a
                left join (
                    select
                        dt,
                        city_id,
                        case
                            when product_name = '快车' then '普通快车'
                            else product_name
                        end as product_name,
                        total_tsh
                    from
                        t1
                    where
                        not product_name = '普通快车'
                ) b on a.dt = b.dt
                and a.city_id = b.city_id
                and a.product_name = b.product_name
        ) aa
        left join (
            select
                dt,
                city_id,
                product_name,
                charge_dur
            from
                t1
        ) bb on aa.dt = bb.dt
        and aa.city_id = bb.city_id
        and aa.src_product_name = bb.product_name
),
t2 as (
    select
        aa.city_id as city_id,
        morning_peak_bubble_cnt,
        evening_peak_bubble_cnt,
        night_peak_bubble_cnt,
        morning_peak_call_cnt,
        evening_peak_call_cnt,
        night_peak_call_cnt,
        lineup_call_cnt,
        morning_peak_lineup_call_cnt,
        evening_peak_lineup_call_cnt,
        night_peak_lineup_call_cnt,
        est_dur,
        est_arrive_dur,
        morning_peak_est_arrive_dur,
        evening_peak_est_arrive_dur,
        night_peak_est_arrive_dur,
        est_arrive_dis,
        morning_peak_success_cnt,
        evening_peak_success_cnt,
        night_peak_success_cnt,
        morning_peak_gmv,
        evening_peak_gmv,
        night_peak_gmv,
        long_dis_success_cnt,
        short_dis_success_cnt,
        long_dis_gmv,
        short_dis_gmv,
        aa.dt as dt,
        aa.level_1_product as level_1_product,
        aa.level_2_product as level_2_product,
        aa.level_3_product as level_3_product
    from
        (
            select
                a.city_id as city_id,
                morning_peak_bubble_cnt,
                evening_peak_bubble_cnt,
                night_peak_bubble_cnt,
                morning_peak_call_cnt,
                evening_peak_call_cnt,
                night_peak_call_cnt,
                lineup_call_cnt,
                morning_peak_lineup_call_cnt,
                evening_peak_lineup_call_cnt,
                night_peak_lineup_call_cnt,
                est_dur,
                est_arrive_dur,
                morning_peak_est_arrive_dur,
                evening_peak_est_arrive_dur,
                night_peak_est_arrive_dur,
                est_arrive_dis,
                a.dt as dt,
                a.level_1_product as level_1_product,
                a.level_2_product as level_2_product,
                a.level_3_product as level_3_product
            from
                (
                    select
                        city_id,
                        count(
                            if(
                                bubble_hour >= 07
                                and bubble_hour < 10,
                                bubble_hour,
                                null
                            )
                        ) as morning_peak_bubble_cnt,
                        --早高峰冒泡数
                        count(
                            if(
                                bubble_hour >= 17
                                and bubble_hour < 20,
                                bubble_hour,
                                null
                            )
                        ) as evening_peak_bubble_cnt,
                        --晚高峰冒泡数
                        count(
                            if(
                                bubble_hour >= 21
                                or bubble_hour < 5,
                                bubble_hour,
                                null
                            )
                        ) as night_peak_bubble_cnt,
                        --夜高峰冒泡数
                        dt,
                        level_1_product,
                        level_2_product,
                        level_3_product
                    from
                        (
                            select
                                dt,
                                bubble_id,
                                start_city_id as city_id,
                                substring(bubble_time, 12, 2) as bubble_hour
                            from
                                dd_cdm.dwd_trd_bubble_distinct_di
                            where
                                dt = '2023-04-01'
                                and level1_product_id in (110000, 120000)
                                and city_id in (1, 42, 366)
                        ) a
                        left join(
                            select
                                distinct bubble_id,
                                level_1_product,
                                level_2_product,
                                level_3_product
                            from
                                dd_cdm.dwd_trip_trd_bubble_anycar_di
                            where
                                dt = '2023-04-01'
                                and level_1_product in (110000, 120000)
                                and city_id in (1, 42, 366)
                        ) b on a.bubble_id = b.bubble_id
                    group by
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product
                ) a full
                join (
                    select
                        city_id,
                        count(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                call_hour,
                                null
                            )
                        ) as morning_peak_call_cnt,
                        --早高峰发单数
                        count(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                call_hour,
                                null
                            )
                        ) as evening_peak_call_cnt,
                        --晚高峰发单数
                        count(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                call_hour,
                                null
                            )
                        ) as night_peak_call_cnt,
                        --夜高峰发单数
                        sum(is_lineup) as lineup_call_cnt,
                        --排队订单数
                        sum(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                is_lineup,
                                0
                            )
                        ) as morning_peak_lineup_call_cnt,
                        --早高峰排队订单数
                        sum(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                is_lineup,
                                0
                            )
                        ) as evening_peak_lineup_call_cnt,
                        --晚高峰排队订单数
                        sum(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                is_lineup,
                                0
                            )
                        ) as night_peak_lineup_call_cnt,
                        --夜高峰排队订单数
                        sum(est_dur) as est_dur,
                        --预估时长
                        sum(est_arrive_dur) as est_arrive_dur,
                        --预计接驾时长
                        sum(
                            if(
                                call_hour >= 07
                                and call_hour < 10,
                                est_arrive_dur,
                                0
                            )
                        ) as morning_peak_est_arrive_dur,
                        --早高峰预计接驾时长
                        sum(
                            if(
                                call_hour >= 17
                                and call_hour < 20,
                                est_arrive_dur,
                                0
                            )
                        ) as evening_peak_est_arrive_dur,
                        --晚高峰预计接驾时长
                        sum(
                            if(
                                call_hour >= 21
                                or call_hour < 5,
                                est_arrive_dur,
                                0
                            )
                        ) as night_peak_est_arrive_dur,
                        --夜高峰预计接驾时长
                        sum(est_arrive_dis) as est_arrive_dis,
                        --预计接驾距离
                        dt,
                        level_1_product,
                        level_2_product,
                        level_3_product
                    from
                        (
                            select
                                dt,
                                call_city as city_id,
                                level_1_product,
                                level_2_product,
                                level_3_product,
                                substring(call_time, 12, 2) as call_hour,
                                is_lineup,
                                est_dur,
                                est_arrive_dur,
                                est_arrive_dis
                            from
                                decdm.dwd_gflt_ord_order_base_di
                            where
                                dt = '2023-04-01'
                                and level_1_product in (110000, 120000)
                                and city_id in (1, 42, 366)
                        ) t
                    group by
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product
                ) b on a.dt = b.dt
                and a.city_id = b.city_id
                and a.level_1_product = b.level_1_product
                and a.level_2_product = b.level_2_product
                and a.level_3_product = b.level_3_product
        ) aa full
        join (
            select
                city_id,
                count(
                    if(
                        end_charge_hour >= 07
                        and end_charge_hour < 10,
                        end_charge_hour,
                        null
                    )
                ) as morning_peak_success_cnt,
                --早高峰完单数
                count(
                    if(
                        end_charge_hour >= 17
                        and end_charge_hour < 20,
                        end_charge_hour,
                        null
                    )
                ) as evening_peak_success_cnt,
                --晚高峰完单数
                count(
                    if(
                        end_charge_hour >= 21
                        or end_charge_hour < 5,
                        end_charge_hour,
                        null
                    )
                ) as night_peak_success_cnt,
                --夜高峰完单数
                sum(
                    if(
                        end_charge_hour >= 07
                        and end_charge_hour < 10,
                        gmv_amt / 100,
                        0
                    )
                ) as morning_peak_gmv,
                --早高峰完单GMV
                sum(
                    if(
                        end_charge_hour >= 17
                        and end_charge_hour < 20,
                        gmv_amt,
                        0
                    )
                ) as evening_peak_gmv,
                --晚高峰完单GMV
                sum(
                    if(
                        end_charge_hour >= 21
                        or end_charge_hour < 5,
                        gmv_amt,
                        0
                    )
                ) as night_peak_gmv,
                --夜高峰完单GMV
                count(
                    if(
                        est_dis >= 15000,
                        est_dis,
                        null
                    )
                ) as long_dis_success_cnt,
                --长距离完单数
                count(
                    if(
                        est_dis < 15000,
                        est_dis,
                        null
                    )
                ) as short_dis_success_cnt,
                --短距离完单数
                sum(
                    if(
                        est_dis >= 15000,
                        gmv_amt / 100,
                        0
                    )
                ) as long_dis_gmv,
                --长距离完单GMV
                sum(
                    if(
                        est_dis < 15000,
                        gmv_amt / 100,
                        0
                    )
                ) as short_dis_gmv,
                --短距离完单GMV
                dt,
                level_1_product,
                level_2_product,
                level_3_product
            from
                (
                    select
                        dt,
                        city_id,
                        level_1_product,
                        level_2_product,
                        level_3_product,
                        substring(end_charge_time, 12, 2) as end_charge_hour,
                        est_dis,
                        gmv_amt
                    from
                        dd_cdm.dwd_trip_trd_attribute_call_order_base_di
                    where
                        dt = '2023-04-01'
                        and level_1_product in (110000, 120000)
                        and city_id in (1, 42, 366)
                ) t
            group by
                dt,
                city_id,
                level_1_product,
                level_2_product,
                level_3_product
        ) bb on aa.dt = bb.dt
        and aa.city_id = bb.city_id
        and aa.level_1_product = bb.level_1_product
        and aa.level_2_product = bb.level_2_product
        and aa.level_3_product = bb.level_3_product
),
order_table as (
    select
        city_id,
        dt,
        case
            when level_3_product = 110103 then '特惠快车'
            when level_3_product = 110101 then '普通快车'
        end as product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_3_product in (110103, 110101)
    group by
        city_id,
        dt,
        case
            when level_3_product = 110103 then '特惠快车'
            when level_3_product = 110101 then '普通快车'
        end
    union
    select
        city_id,
        dt,
        case
            when level_2_product = 110200 then '市内拼车'
            when level_2_product = 110500 then '专车'
        end as product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_2_product in (110200, 110500)
    group by
        city_id,
        dt,
        case
            when level_2_product = 110200 then '市内拼车'
            when level_2_product = 110500 then '专车'
        end
    union
    select
        city_id,
        dt,
        case
            when level_1_product = 120000 then '出租车'
            when level_1_product = 110000 then '网约车'
        end as product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_1_product in (120000, 110000)
    group by
        city_id,
        dt,
        case
            when level_1_product = 120000 then '出租车'
            when level_1_product = 110000 then '网约车'
        end
    union
    select
        city_id,
        dt,
        '网+出' product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_1_product in (120000, 110000)
    group by
        city_id,
        dt
    union
    select
        city_id,
        dt,
        '新泛快' as product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_3_product in (110103, 110101, 110102)
    group by
        city_id,
        dt
    union
    select
        city_id,
        dt,
        '泛快大盘' as product_name,
        sum(morning_peak_bubble_cnt) as morning_peak_bubble_cnt,
        sum(evening_peak_bubble_cnt) as evening_peak_bubble_cnt,
        sum(night_peak_bubble_cnt) as night_peak_bubble_cnt,
        sum(morning_peak_call_cnt) as morning_peak_call_cnt,
        sum(evening_peak_call_cnt) as evening_peak_call_cnt,
        sum(night_peak_call_cnt) as night_peak_call_cnt,
        sum(lineup_call_cnt) as lineup_call_cnt,
        sum(morning_peak_lineup_call_cnt) as morning_peak_lineup_call_cnt,
        sum(evening_peak_lineup_call_cnt) as evening_peak_lineup_call_cnt,
        sum(night_peak_lineup_call_cnt) as night_peak_lineup_call_cnt,
        sum(est_dur) as est_dur,
        sum(est_arrive_dur) as est_arrive_dur,
        sum(morning_peak_est_arrive_dur) as morning_peak_est_arrive_dur,
        sum(evening_peak_est_arrive_dur) as evening_peak_est_arrive_dur,
        sum(night_peak_est_arrive_dur) as night_peak_est_arrive_dur,
        sum(est_arrive_dis) as est_arrive_dis,
        sum(morning_peak_success_cnt) as morning_peak_success_cnt,
        sum(evening_peak_success_cnt) as evening_peak_success_cnt,
        sum(night_peak_success_cnt) as night_peak_success_cnt,
        sum(morning_peak_gmv) as morning_peak_gmv,
        sum(evening_peak_gmv) as evening_peak_gmv,
        sum(night_peak_gmv) as night_peak_gmv,
        sum(long_dis_success_cnt) as long_dis_success_cnt,
        sum(short_dis_success_cnt) as short_dis_success_cnt,
        sum(long_dis_gmv) as long_dis_gmv,
        sum(short_dis_gmv) as short_dis_gmv
    from
        t2
    where
        level_2_product in (110100, 110200, 110400, 110800, 110900)
    group by
        city_id,
        dt
),
t3 as(
    select
        area_id as city_id,
        count(if(exp_discount < 70, exp_discount, null)) as discount_00_to_70_cnt,
        count(
            if(
                exp_discount >= 70
                and exp_discount < 80,
                exp_discount,
                null
            )
        ) as discount_70_to_80_cnt,
        count(
            if(
                exp_discount >= 80
                and exp_discount < 90,
                exp_discount,
                null
            )
        ) as discount_80_to_90_cnt,
        count(
            if(
                exp_discount >= 90
                and exp_discount < 95,
                exp_discount,
                null
            )
        ) as discount_90_to_95_cnt,
        count(
            if(
                exp_discount >= 95
                and exp_discount <= 100,
                exp_discount,
                null
            )
        ) as discount_95_to_100_cnt,
        dt,
        current_product_type
    from
        mp_data.dwd_trip_mp_hufan_bubble_di
    where
        dt = '2023-04-01'
        and current_product_type in (
            206,
            2061,
            210,
            208,
            315,
            209,
            100,
            400,
            900,
            314,
            2200,
            500
        )
        and city_id in (1, 42, 366)
    group by
        dt,
        area_id,
        current_product_type
),
discount_table as (
    select
        city_id,
        dt,
        case
            when current_product_type = 314 then '特惠快车'
            when current_product_type = 206 then '普通快车'
            when current_product_type in (100, 400) then '专车'
            when current_product_type = 500 then '出租车'
        end as product_name,
        sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
        sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
        sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
        sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
        sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
    from
        t3
    where
        current_product_type in (314, 206, 100, 400, 500)
    group by
        dt,
        city_id,
        case
            when current_product_type = 314 then '特惠快车'
            when current_product_type = 206 then '普通快车'
            when current_product_type in (100, 400) then '专车'
            when current_product_type = 500 then '出租车'
        end
    union
    select
        city_id,
        dt,
        '网约车' product_name,
        sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
        sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
        sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
        sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
        sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
    from
        t3
    where
        current_product_type in (
            206,
            2061,
            210,
            208,
            315,
            209,
            100,
            400,
            900,
            314,
            2200
        )
    group by
        dt,
        city_id
    union
    select
        city_id,
        dt,
        '网+出' product_name,
        sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
        sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
        sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
        sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
        sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
    from
        t3
    where
        current_product_type in (
            206,
            2061,
            210,
            208,
            315,
            209,
            100,
            400,
            900,
            314,
            2200,
            500
        )
    group by
        dt,
        city_id
    union
    select
        city_id,
        dt,
        '新泛快' product_name,
        sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
        sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
        sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
        sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
        sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
    from
        t3
    where
        current_product_type in (206, 2061, 314)
    group by
        dt,
        city_id
    union
    select
        city_id,
        dt,
        '泛快大盘' product_name,
        sum(discount_00_to_70_cnt) as discount_00_to_70_cnt,
        sum(discount_70_to_80_cnt) as discount_70_to_80_cnt,
        sum(discount_80_to_90_cnt) as discount_80_to_90_cnt,
        sum(discount_90_to_95_cnt) as discount_90_to_95_cnt,
        sum(discount_95_to_100_cnt) as discount_95_to_100_cnt
    from
        t3
    where
        current_product_type in (206, 2061, 210, 208, 315, 209, 314)
    group by
        dt,
        city_id
)
select
    passenger_table.city_id,
    --城市
    passenger_table.bubble_cnt,
    --冒泡数
    passenger_table.high_freq_bubble_cnt,
    --中高活跃用户冒泡数
    passenger_table.low_freq_bubble_cnt,
    --低活跃用户冒泡数
    passenger_table.xcl_freq_bubble_cnt,
    --新沉流用户冒泡数
    order_table.morning_peak_bubble_cnt,
    --早高峰冒泡数
    order_table.evening_peak_bubble_cnt,
    --晚高峰冒泡数
    order_table.night_peak_bubble_cnt,
    --夜高峰冒泡数
    discount_table.discount_00_to_70_cnt,
    --7折以下订单数
    discount_table.discount_70_to_80_cnt,
    --7折到8折订单数
    discount_table.discount_80_to_90_cnt,
    --8折到9折订单数
    discount_table.discount_90_to_95_cnt,
    --9折到95折订单数
    discount_table.discount_95_to_100_cnt,
    --95折以上订单数
    passenger_table.call_cnt,
    --发单数
    passenger_table.high_freq_call_cnt,
    --中高活跃用户发单数
    passenger_table.low_freq_call_cnt,
    --低活跃用户发单数
    passenger_table.xcl_freq_call_cnt,
    --新沉流用户发单数
    order_table.morning_peak_call_cnt,
    --早高峰发单数
    order_table.evening_peak_call_cnt,
    --晚高峰发单数
    order_table.night_peak_call_cnt,
    --夜高峰发单数
    order_table.lineup_call_cnt,
    --排队订单数
    order_table.morning_peak_lineup_call_cnt,
    --早高峰排队订单数
    order_table.evening_peak_lineup_call_cnt,
    --晚高峰排队订单数
    order_table.night_peak_lineup_call_cnt,
    --夜高峰排队订单数
    order_table.est_dur,
    --预估时长
    order_table.est_arrive_dur,
    --预计接驾时长
    order_table.morning_peak_est_arrive_dur,
    --早高峰预计接驾时长
    order_table.evening_peak_est_arrive_dur,
    --晚高峰预计接驾时长
    order_table.night_peak_est_arrive_dur,
    --夜高峰预计接驾时长
    order_table.est_arrive_dis,
    --预计接驾距离
    city_table.total_tsh,
    --TSH（没有普通快车、市内拼车，使用快车替代普通快车）
    city_table.th_yj_tsh,
    --愿接特惠时长
    city_table.answer_order_cnt,
    --应答数
    --计费时长（没有普通快车、市内拼车、特惠快车，使用快车替代普通快车和特惠快车）
    --TPH = 完单数 / TSH
    --供需比 = TSH / 发单数
    --需供比 = 发单数 / TSH
    --应答率 = 应答数 / 发单数
    passenger_table.success_cnt,
    --完单数
    passenger_table.high_freq_success_cnt,
    --中高活跃用户完单数
    passenger_table.low_freq_success_cnt,
    --低活跃用户完单数
    passenger_table.xcl_freq_success_cnt,
    --新沉流用户完单数
    order_table.morning_peak_success_cnt,
    --早高峰完单数
    order_table.evening_peak_success_cnt,
    --晚高峰完单数
    order_table.night_peak_success_cnt,
    --夜高峰完单数
    order_table.long_dis_success_cnt,
    --长距离完单数
    order_table.short_dis_success_cnt,
    --短距离完单数
    passenger_table.gmv,
    --完单GMV
    passenger_table.high_freq_gmv,
    --中高活跃用户完单GMV
    passenger_table.low_freq_gmv,
    --低活跃用户完单GMV
    passenger_table.xcl_freq_gmv,
    --新沉流用户完单GMV
    order_table.morning_peak_gmv,
    --早高峰完单GMV
    order_table.evening_peak_gmv,
    --晚高峰完单GMV
    order_table.night_peak_gmv,
    --夜高峰完单GMV
    order_table.long_dis_gmv,
    --长距离完单GMV
    order_table.short_dis_gmv,
    --短距离完单GMV
    passenger_table.driver_divide,
    --司机抽成
    passenger_table.business_profit,
    --毛利额
    passenger_table.subsidy_hufan,
    --完单呼返
    passenger_table.subsidy_c,
    --完单C补
    passenger_table.subsidy_b,
    --完单B补
    passenger_table.dt,
    --日期
    passenger_table.product_name --品类名称
from
    passenger_table full
    join city_table on passenger_table.dt = city_table.dt
    and passenger_table.city_id = city_table.city_id
    and passenger_table.product_name = city_table.product_name full
    join order_table on city_table.dt = order_table.dt
    and city_table.city_id = order_table.city_id
    and city_table.product_name = order_table.product_name full
    join discount_table on order_table.dt = discount_table.dt
    and order_table.city_id = discount_table.city_id
    and order_table.product_name = discount_table.product_name
order by
    passenger_table.city_id, passenger_table.product_name
limit
    100
''').toPandas()
df

,city_id,bubble_cnt,high_freq_bubble_cnt,low_freq_bubble_cnt,xcl_freq_bubble_cnt,morning_peak_bubble_cnt,evening_peak_bubble_cnt,night_peak_bubble_cnt,discount_00_to_70_cnt,discount_70_to_80_cnt,discount_80_to_90_cnt,discount_90_to_95_cnt,discount_95_to_100_cnt,call_cnt,high_freq_call_cnt,low_freq_call_cnt,xcl_freq_call_cnt,morning_peak_call_cnt,evening_peak_call_cnt,night_peak_call_cnt,lineup_call_cnt,morning_peak_lineup_call_cnt,evening_peak_lineup_call_cnt,night_peak_lineup_call_cnt,est_dur,est_arrive_dur,morning_peak_est_arrive_dur,evening_peak_est_arrive_dur,night_peak_est_arrive_dur,est_arrive_dis,total_tsh,th_yj_tsh,answer_order_cnt,success_cnt,high_freq_success_cnt,low_freq_success_cnt,xcl_freq_success_cnt,morning_peak_success_cnt,evening_peak_success_cnt,night_peak_success_cnt,long_dis_success_cnt,short_dis_success_cnt,gmv,high_freq_gmv,low_freq_gmv,xcl_freq_gmv,morning_peak_gmv,evening_peak_gmv,night_peak_gmv,long_dis_gmv,short_dis_gmv,driver_divide,business_profit,subsidy_hufan,subsidy_c,subsidy_b,dt,product_name
0,1,1771138.000000,1085672.000000,367967.000000,317499.000000,4598109.0,4135859.0,4671076.0,1032325.0,600852.0,1647011.0,32523.0,12845411.0,164390.000000,132564.000000,21125.000000,10701.000000,16239.0,25516.0,36216.0,8579.0,1456.0,661.0,3488.0,218777160.000000,59702098.000000,7615872.000000,10081336.000000,13853086.000000,388647574.000000,96382.133333,0.000000,136375.0,113816.000000,93481.000000,14019.000000,6316.000000,10783.0,23626.0,51765.0,39033.0,102420.0,7807245.920000,6347142.350000,992032.700000,468070.870000,733159.4800000000,153547731.000000,215246767.000000,4378904.7900000000,3751494.6300000000,6028287.150000,1156209.630413,206227.330000,288016.849400,21742.090000,2023-04-01,专车
1,1,2436948.000000,1530592.000000,512515.000000,393841.000000,2351105.0,2157457.0,2454474.0,NaN,NaN,NaN,NaN,NaN,355222.000000,246211.000000,68039.000000,40972.000000,53604.0,40136.0,47993.0,13097.0,3180.0,883.0,2577.0,455420220.000000,73463389.000000,13491347.000000,10441279.000000,13237284.000000,455175394.000000,192287.150000,0.000000,259808.0,223737.000000,158186.000000,42243.000000,23308.000000,41058.0,35904.0,72352.0,74960.0,198113.0,11875845.954838,7788524.359284,2219587.435839,1867734.159715,1553877.3389710000,132935145.042900,174811951.829600,4917675.3675670000,4222397.9850870000,9135201.088728,-54297.427793,0.000000,56553.700000,2937.984881,2023-04-01,出租车
2,1,2066262.000000,1404049.000000,387289.000000,274924.000000,1019709.0,1131155.0,598344.0,NaN,NaN,NaN,NaN,NaN,219398.000000,166239.000000,33409.000000,19750.000000,40654.0,42031.0,22908.0,51324.0,12442.0,6454.0,3655.0,287165340.000000,48889845.000000,10205424.000000,9404193.000000,4661180.000000,284200934.000000,0.000000,0.000000,169405.0,146852.000000,111548.000000,22710.000000,12594.000000,24705.0,32423.0,76127.0,50179.0,159053.0,4823328.090000,3670827.450000,730346.740000,422153.900000,695967.4500000000,100723226.000000,73577815.000000,2054289.6500000000,2801734.0800000000,2938175.457129,270675.840674,0.000000,1321409.828800,138810.350624,2023-04-01,市内拼车
3,1,6907676.000000,4350575.000000,1417840.000000,1139261.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1385945.000000,960916.000000,259813.000000,165216.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,NaN,983561.000000,699079.000000,179703.000000,104779.000000,NaN,NaN,NaN,NaN,NaN,34665973.540000,24612260.920000,6272121.530000,3781591.090000,None,None,None,None,None,26311947.620521,3573618.078397,793039.660000,2924592.941800,533447.416010,2023-04-01,新泛快
4,1,3054183.000000,1919001.000000,625095.000000,510087.000000,2470041.0,2278228.0,2593748.0,0.0,61532.0,368920.0,7008.0,5488069.0,967760.000000,665708.000000,181312.000000,120740.000000,133613.0,151993.0,206537.0,229465.0,34640.0,28370.0,37963.0,1194805140.000000,238604949.000000,35518977.000000,33646279.000000,48522001.000000,1494964495.000000,611404.000000,0.000000,807816.0,681373.000000,480685.000000,124860.000000,75828.000000,88697.0,13